## Usage
Make sure azure_notebook_reporting is installed as a first step, run below using a code cell once off

```python
%pip install --quiet git+https://github.com/wagov/wasoc-notebook.git
```

To run in Azure ML
```python
import os, json, pandas as pd
from azure_notebook_reporting import KQL, BlobPath
os.environ.update(json.load(open(f"{os.environ['HOME']}/cloudfiles/code/nbenv.json")))
path = BlobPath(os.environ["AZURE_STORAGE_CONTAINER"], os.environ["AZURE_SUBSCRIPTION"])
kp = KQL(path, template="markdown/report-sentinel.md")
```

To run locally
```python
import os, json, pandas as pd
from azure_notebook_reporting import KQL, BlobPath
path = BlobPath("..")
kp = KQL(path, template="markdown/report-sentinel.md")
```

In [ ]:
import os, json, hashlib, pandas as pd
from azure_notebook_reporting import KQL, BlobPath

%matplotlib agg

path = BlobPath("..")
template = "markdown/report-sentinel.md"
queries = {
    "Users and Azure AD Logins": "siemhealth/signins.kql",
    "Email Delivery": "siemhealth/emaildelivery.kql",
    "Incident Detection & Response": "siemhealth/incidentdetail.kql",
    "On Premise Logins": "siemhealth/identitylogonevents.kql",
    "External Files": "siemhealth/externaldownloads.kql",
    "Guest Tenants": "siemhealth/guestdownloads.kql",
    "Operating Systems": "siemhealth/operatingsystems.kql",
    "Admin Logins (device)": "siemhealth/localadminlogons.kql",
    "Ingestion Detail": "siemhealth/usage.kql",
}

kp = KQL(path, template)
kp.set_agency(os.environ["REPORT_AGENCY"], sample_agency=os.environ["REPORT_SAMPLE_AGENCY"], sample_only=False)
kp.load_queries(queries)

kp.init_report(
    background = "https://raw.githubusercontent.com/wagov/wasoc-notebook/main/notebooks/background.svg",
    entity = kp.agency_name,
    date = kp.today.strftime("%B %Y"),
    body = "#121212",
    links = "#084595",
    titles = "#CC5733",
    footer = "#808080"
)

rp = kp.report

section = "Executive Summary"
exec_summary = kp.nbpath / f"markdown/exec_summaries/{kp.agency}-{kp.today.strftime('%b%Y')}.md"
if not exec_summary.exists():
    exec_summary = kp.nbpath / "markdown/exec_summaries/default.md"

rp[section] = kp.report_sections["Executive Summary"].substitute(
    agency = kp.agency_name,
    date = kp.today.strftime("%B %Y"),
    summary = exec_summary.open().read()
)

# Workaround for table name being incorrect
if kp.querystats["Rows"]["Operating Systems"] == 0:
    kp.querystats["Columns"]["Operating Systems"] = "DeviceEvents - No Data in timespan P30D"

rp[section] += kp.querystats
kp.show(section)

## Composing a report

Setup each section, loading markdown fragments from the kp.report_sections dict and dataframes from the kp.queries dict

In [ ]:
section = "Users and Azure AD Logins"
signins = kp.queries[section][1]
distinct_users = signins["UserPrincipalName"].count()
rp[section] = kp.report_sections[section].substitute(users=distinct_users)

if kp.querystats["Rows"][section] == 0:
    rp[section] = "*MISSING DATA: Please verify the [Microsoft Defender for Endpoint connection](https://learn.microsoft.com/en-us/azure/sentinel/connect-microsoft-365-defender?tabs=MDE).*"
    table = kp.querystats["Columns"][section].split(" - ")[0]
    rp[section] += f"### SAMPLE DATASET BELOW: Sentinel Table {table}"
    rp[section] += kp.report_sections[section].substitute(users=distinct_users)


df = signins.replace(0, None).rename(columns={"TableName": "SignIns"}).groupby("SignIns").count()
df = df.T.rename(columns={"PrimaryResult": "Percent of Users"}).sort_values("Percent of Users").drop("UserPrincipalName")
df = df / distinct_users * 100
rp[section] += df.plot(kind="barh", title="Azure AD SignIn types over past 30 days").figure
kp.show(section)

In [ ]:
section = "Email Delivery"
rp[section] = kp.report_sections[section].substitute()
emails = kp.queries[section][1]

if kp.querystats["Rows"][section] == 0:
    rp[section] = "*MISSING DATA: Please verify the [Microsoft Defender for Office 365 connection](https://learn.microsoft.com/en-us/microsoft-365/security/office-365-security/step-by-step-guides/connect-microsoft-defender-for-office-365-to-microsoft-sentinel?view=o365-worldwide)*"
    table = kp.querystats["Columns"][section].split(" - ")[0]
    rp[section] += f"### SAMPLE DATASET BELOW: Sentinel Table {table}"
    rp[section] += kp.report_sections[section].substitute()


df = emails.groupby(["DeliveryAction", "EmailDirection"]).agg({"Count": "sum"})
df["Count"] = df["Count"] / 1000
df = df.unstack("EmailDirection").sort_values(("Count", "Inbound"), ascending=False)
rp[section] += df.plot(kind="barh", y="Count", stacked=True, title="Total email delivered (thousands) in the past 30 days").figure

df = emails.groupby(["EmailDirection", "DeliveryAction"]).resample("6H", on="TimeGenerated").agg({"Count": "sum"}).sort_values("Count").unstack("DeliveryAction").unstack("EmailDirection")
rp[section] += df.plot(kind="area", y="Count", colormap=KQL.sns.color_palette("viridis", as_cmap=True), title="Email Delivery over past 30 days, 6 hour intervals").figure

df = KQL.latest_data(emails, "7D")
df = df.groupby(["EmailDirection", "DeliveryAction"]).resample("1H", on="TimeGenerated").agg({"Count": "sum"}).sort_values("Count").unstack("DeliveryAction").unstack("EmailDirection")
rp[section] += df.plot(kind="area", y="Count", colormap=KQL.sns.color_palette("viridis", as_cmap=True), title="Email Delivery over past 7 days, 1 hour intervals").figure

kp.show(section)

In [ ]:
section = "Incident Detection & Response"
rp[section] = kp.report_sections[section].substitute()
incidents = kp.queries[section][1]
incidents["TriageHours"] = pd.to_numeric(incidents["TriageHours"].replace({"None": '0'}))

if kp.querystats["Rows"][section] == 0:
    rp[section] = "*MISSING DATA: Please confirm there are [analytics rules](https://learn.microsoft.com/en-us/azure/sentinel/detect-threats-built-in) configured on the Microsoft Sentinel > Analytics > Rule templates page.*"
    table = kp.querystats["Columns"][section].split(" - ")[0]
    rp[section] += f"### SAMPLE DATASET BELOW: Sentinel Table {table}"
    rp[section] += kp.report_sections[section].substitute()
    KQL.hash_columns(incidents, "Rule")

    
df = incidents.groupby(["Status", "Classification", "Severity", "Tactics", "Rule"])[["IncidentNumber", "TriageHours", "OpenHours"]]
df = df.agg({"IncidentNumber": "nunique", "TriageHours": "sum", "OpenHours": "sum"})
df["TriageHours"] = df["TriageHours"] / df["IncidentNumber"]
df["OpenHours"] = df["OpenHours"] / df["IncidentNumber"]
df.groupby(["Status", "Classification", "Severity", "Tactics", "Rule"]).agg('sum')
df = df.rename(columns={"IncidentNumber": "Incidents"}).convert_dtypes()
df = df.convert_dtypes().sort_values("Incidents", ascending=False)

rp[section] += "### Top 10 rules creating incidents over past 30 days"
rp[section] += df.head(10).round(2)

noisyrule = ""
if (df["Incidents"][0].sum() / df["Incidents"][1].sum()) > 5:
    df, noisyrule = df[1:], df.index.values[0][4]
    noisyrule = f' excluding outlier "{noisyrule}"'

rp[section] += df["TriageHours"].head(10).unstack(level=[0,1,2]).plot(kind="barh", stacked=False, width=1, title=f'Average Triage Hours over last 30 days{noisyrule}').figure
rp[section] += df["OpenHours"].head(10).unstack(level=[0,1,2]).plot(kind="barh", stacked=False, width=1, title=f'Average Open Hours over last 30 days{noisyrule}').figure
rp[section] += df["Incidents"].head(10).unstack(level=[0,1,2]).plot(kind="barh", stacked=False, width=1, title=f'Average Detections over last 30 days{noisyrule}').figure
kp.show(section)

In [ ]:
section = "On Premise Logins"
onpremsignins = kp.queries[section][1]
df = onpremsignins[onpremsignins["dcs"] != '[""]']
accounts, devices, dcs = df["users"].max(), df["devices"].max(), df["dcs"].apply(lambda l: len(l.split(","))).max()

if kp.querystats["Rows"][section] == 0:
    rp[section] = "*MISSING DATA: Please verify the [Microsoft Defender for Identity](https://learn.microsoft.com/en-us/azure/sentinel/connect-microsoft-365-defender?tabs=MDI) connection*"
    table = kp.querystats["Columns"][section].split(" - ")[0]
    rp[section] += f"### SAMPLE DATASET BELOW: Sentinel Table {table}"
    rp[section] += kp.report_sections[section].substitute(accounts = accounts, devices = devices, dcs = dcs)
    KQL.hash_columns(df, "dcs")

df = df.drop("TableName", axis=1)[["logontype", "users", "devices", "dcs"]].set_index("logontype")

rp[section] += df
df["users"] = df["users"] / accounts * 100
df["devices"] = df["devices"] / devices * 100
rp[section] += df.plot(kind="barh", title=f"Logon types by percentage over past 30 days ({accounts} users signed in to {devices} devices using {dcs} dcs)").figure

kp.show(section)

In [ ]:
section = "Operating Systems"
platforms = kp.queries[section][1]
asr, devs = platforms["ASREnabled"].sum(), platforms["devices"].sum()
rp[section] = kp.report_sections[section].substitute(asr = asr, devs = devs, devices = devices)

if kp.querystats["Rows"][section] == 0:
    rp[section] = "*MISSING DATA: Please verify the [Defender for Endpoint](https://learn.microsoft.com/en-us/azure/sentinel/connect-microsoft-365-defender?tabs=MDE#connect-to-microsoft-365-defender) connection.*"
    table = kp.querystats["Columns"][section].split(" - ")[0]
    rp[section] += f"### SAMPLE DATASET BELOW: Sentinel Table {table}"
    rp[section] += kp.report_sections[section].substitute(asr = asr, devs = devs, devices = devices)
    

df = platforms.drop("TableName", axis=1).set_index("OSPlatform")
rp[section] += df.plot(kind="barh").figure
rp[section] += df
kp.show(section)

In [ ]:
section = "Admin Logins (device)"
admins = kp.queries[section][1]
admincount = admins[admins["Devices"] > 5]["AccountName"].nunique()
rp[section] = kp.report_sections[section].substitute(admincount = admincount)

if kp.querystats["Rows"][section] == 0:
    rp[section] = "*MISSING DATA: Please verify the [Defender for Endpoint](https://learn.microsoft.com/en-us/azure/sentinel/connect-microsoft-365-defender?tabs=MDE#connect-to-microsoft-365-defender) connection.*"
    table = kp.querystats["Columns"][section].split(" - ")[0]
    rp[section] += f"### SAMPLE DATASET BELOW: Sentinel Table {table}"
    rp[section] += kp.report_sections[section].substitute(admincount = admincount)
    KQL.hash_columns(admins, "AccountName")
    

rp[section] += admins.head(10).plot(kind="barh",x="AccountName", title="Top 10 admin user signins over past 30 days").figure
kp.show(section)

In [ ]:
section = "Office 365 Activity"
rp[section] = kp.report_sections[section].substitute()
spfiles = kp.queries["External Files"][1]
tenants = kp.queries["Guest Tenants"][1]

if kp.querystats["Rows"]["External Files"] == 0:
    rp[section] = "*MISSING DATA: Please verify the [Defender for Office 365](https://learn.microsoft.com/en-us/azure/sentinel/connect-microsoft-365-defender?tabs=MDO#connect-to-microsoft-365-defender) connection.*"
    table = kp.querystats["Columns"]["External Files"].split(" - ")[0]
    rp[section] += f"### SAMPLE DATASET BELOW: Sentinel Table {table}"
    KQL.hash_columns(spfiles, "FileUrl")
    KQL.hash_columns(tenants, "Guest Domain")
    rp[section] += kp.report_sections[section].substitute()

rp[section] += "### Externally accessed sharepoint files (top 10)"
rp[section] += spfiles.head(10).iloc[:,:-1].set_index("FileUrl", drop=True).head(10)

rp[section] += "### Guest domains accessing content (top 10)"
rp[section] += tenants.iloc[:,:-1].set_index("Guest Domain", drop=True).head(10)

kp.show(section)

In [ ]:
section = "Cost Optimisation"
ingestion = kp.queries["Ingestion Detail"][1]
ingestion["GB"] = ingestion["IngestionVolume"].map(lambda v: v / 1000)

df = pd.DataFrame(ingestion.pivot(columns="Table")["GB"].sum())
df["Daily GB"], df["30 days"] = df[0] / 30, df[0]
df = df.drop(0, axis=1).sort_values(by="Daily GB", ascending=False).round(1)
daily, total = int(round(df["Daily GB"].sum(), 0)), int(round(df["30 days"].sum(), 0))
entitled_gb = int(round(distinct_users * 5 / 1000, -1))
high_gb = int(round(distinct_users * 30 / 1000, -1))
rp[section] = kp.report_sections[section].substitute(daily = daily, total = total, licences = distinct_users, entitled_gb = entitled_gb, high_gb = high_gb)
rp[section] += df.head(10)

title = "Top 10 Ingestion tables over the past 30 days, daily average"
rp[section] += pd.DataFrame(df["Daily GB"].head(11)[1:]).plot(kind="barh", title=title, rot=0).figure

df = KQL.latest_data(ingestion, "4D")
df = KQL.label_size(df, "Table", "GB")
df.groupby(["TimeGenerated", "Table"])["GB"].agg("sum").unstack("Table")

oversized_tables = ", ".join(df[df.oversized]["Table"].unique())
title = "Ingestion by table over the past 4 days, 1 hour intervals"
if oversized_tables:
    oversized = df[df.oversized].groupby(["TimeGenerated", "Table"])["GB"].agg("sum").unstack("Table").iloc[:, ::-1]
    rp[section] += oversized.plot(kind="area", stacked=True, title=f"High Volume {title}").figure
    df = df[df.oversized == False]
df = df.groupby(["TimeGenerated", "Table"])["GB"].agg("sum").unstack("Table").iloc[:, ::-1]
rp[section] += df.plot(kind="area", title=title, stacked=True).figure
kp.show(section)

In [ ]:
kp.report_pdf()